In [1]:
import ee 
import os 
import geemap
import geopandas as gpd 
import numpy as np
import time

from glob import glob
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')


In [2]:
gpkg_path = '/media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches'
gpkg_files = glob(f'{gpkg_path}/*.gpkg')
os.listdir(gpkg_path)

['N10_E104_TANDEMX.gpkg',
 'N10_E105_TANDEMX.gpkg',
 'N10_E106_TANDEMX.gpkg',
 'N09_E105_TANDEMX.gpkg',
 'N09_E106_TANDEMX.gpkg']

In [3]:
i = 0
gfile = gpkg_files[i]
g = gpd.read_file(gfile)
g[['minx','miny','maxx','maxy']] = g.bounds

In [4]:
def get_ee_geometry(i, g):
    ig = g.iloc[i:i+1,]
    bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]
    fname = (os.path.basename(ig.location.values[0])).replace('..tif', '_S1.tif')
    region = ee.Geometry.Rectangle(bBox)
    return region, fname

def get_S1mosaic(aoi,pol='VV', opass='ASCENDING',idate='2019-01-01',fdate='2022-12-01'):
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode','IW')) \
    .filterDate(idate,fdate).filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol)) \
    .filter(ee.Filter.eq('orbitProperties_pass',opass)) \
    .filter(ee.Filter.eq('resolution_meters',10)) \
    .filterBounds(aoi)\
    # play with params and see what kind of data you  can get
    s1img = ee.Image(sentinel1.mosaic().clip(aoi))
    
    return s1img


def gee_download_geemap_s1(image,outpath, scale):
    print(outpath)
    if os.path.isfile(outpath):
        print('Already downloaded') 
    else:
        geemap.ee_export_image(image, outpath, scale=scale)

def getS1patch(i,g,pol,S1tile_path,scale):
    region, fname = get_ee_geometry(i, g)
    s1img = get_S1mosaic(region, pol)
    outpath = os.path.join(S1tile_path, fname)
    gee_download_geemap_s1(s1img,outpath, scale)

In [5]:
sentinel1_dir  ='/media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/'
tname = os.path.basename(gfile).replace('_TANDEMX.gpkg','')
S1tile_path = os.path.join(sentinel1_dir, tname)
os.makedirs(S1tile_path, exist_ok=True)

In [40]:
pol = 'VV'
scale = 10
i = 12

getS1patch(i,g,pol,S1tile_path,scale)

/media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/N10_E104/tile_0_3584_S1.tif
Generating URL ...
Please wait ...
Data downloaded to /media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/N10_E104/tile_0_3584_S1.tif


In [16]:
region, fname = get_ee_geometry(i, g)
s1img = get_S1mosaic(region, 'VV')

In [35]:
# band1:VV, VH, angle 

In [17]:
s1img

In [37]:
Map = geemap.Map()
vis_params = {'min': -25, 'max': 0.3, 'gamma': 1.4,}
Map.addLayer(s1img, vis_params, 'S1img')
Map.centerObject(region, 14)
#Map

In [32]:
sentinel1_dir  ='/media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/'
tname = os.path.basename(gfile).replace('_TANDEMX.gpkg','')
S1tile_path = os.path.join(sentinel1_dir, tname)
os.makedirs(S1tile_path, exist_ok=True)

In [33]:

outpath = os.path.join(S1tile_path, fname)
scale=10
outpath

'/media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/N10_E104/tile_0_3072_S1.tif'

In [34]:
gee_download_geemap(s1img,outpath, scale)

/media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/N10_E104/tile_0_3072_S1.tif
Generating URL ...
Please wait ...
Data downloaded to /media/ljp238/6tb/Joseph/DATASETS/SATIMG/SENTINEL1/N10_E104/tile_0_3072_S1.tif


# Sentinel 2

In [41]:
def maskClouds(image):
    qa = image.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

In [60]:
CLOUD_FILTER = 20
def getS2img(region,CLOUD_FILTER=10,idate='2019-01-01',fdate='2022-12-01'):
    s2col = ee.ImageCollection('COPERNICUS/S2_SR') \
              .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))\
             .filterBounds(region) \
             .filterDate(idate,) \
             .map(maskClouds)
    
    s2 = s2col.select('B4', 'B3', 'B2')
    s2img = s2.median().clip(region)
    
    return s2img 

def get_ee_geometry(i, g, name):
    ig = g.iloc[i:i+1,]
    bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]
    fname = (os.path.basename(ig.location.values[0])).replace('..tif', f'_{name}.tif')
    region = ee.Geometry.Rectangle(bBox)
    return region, fname

def getS2collection(region,CLOUD_FILTER=10):
    s2coll = ee.ImageCollection('COPERNICUS/S2_SR') \
             .filterBounds(region) \
             .filterDate('2021', '2022') \
             .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    
    return s2coll 

def getS2_RGBmedian(region,CLOUD_FILTER=10):
    s2coll = ee.ImageCollection('COPERNICUS/S2_SR') \
             .filterBounds(region) \
             .filterDate('2021', '2022') \
             .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    

    sentinel2_masked = s2coll.map(maskClouds)
    rgb_bands = ['B4', 'B3', 'B2']
    rgb = sentinel2_masked.select(rgb_bands).median().clip(region)
    return rgb 

In [ ]:
def getS2RGBpatch(i,g,name,S2tile_path,scale):
    region, fname = get_ee_geometry(i, g,name)
    rgb = getS2_RGBmedian(region)
    outpath = os.path.join(S2tile_path, fname)
    gee_download_geemap(rgb,outpath, scale)
    time.sleep(0.5)


In [ ]:
def getS1patch(i,g,pol,name,S1tile_path,scale):
    #if not os.path.isfile
    region, fname = get_ee_geometry(i, g,name)
    s1img = get_S1mosaic(region, pol)
    outpath = os.path.join(S1tile_path, fname)
    gee_download_geemap_s1(s1img,outpath, scale)
    time.sleep(0.5)

In [61]:
region, fname = get_ee_geometry(i,g,'RGB')
rgb = getS2_RGBmedian(region)


In [62]:
rgb

In [64]:
vis_rgb = {'min': 0, 'max': 0.3, 'gamma': 1.4,}
Map.addLayer(rgb, vis_rgb, 'RGB')
Map.centerObject(region, 12)
Map

Map(bottom=986910.0, center=[10.587555654397661, 104.01422222223083], controls=(WidgetControl(options=['positi…